In [ ]:
!pip install caer canaro

In [ ]:
import os
import canaro
import numpy as np
import caer
import cv2 as cv
import gc

In [ ]:
IMG_SIZE = (80,80)
channels = 1
char_path = r'/kaggle/input/the-simpsons-characters-dataset/simpsons_dataset'

In [ ]:
char_dict = {}
for char in os.listdir(char_path):
    char_dict[char] = len(os.listdir(os.path.join(char_path, char)))

char_dict = caer.sort_dict(char_dict, descending = True)
char_dict

In [ ]:
characters = []
count = 0
for i in char_dict:
    count=count+1
    characters.append(i[0])
    if count >=10:
        break
characters 

In [ ]:
train = caer.preprocess_from_dir(char_path, characters, channels = channels, IMG_SIZE = IMG_SIZE, isShuffle=True)

In [ ]:
len(train)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(30,30))
plt.imshow(train[0][0], cmap = 'gray')
plt.show()

In [ ]:
featureSet, labels = caer.sep_train(train, IMG_SIZE = IMG_SIZE)

In [ ]:
from tensorflow.keras.utils import to_categorical

featureSet = caer.normalize(featureSet)
labels = to_categorical(labels, len(characters))

In [ ]:
x_train, x_val, y_train, y_val = caer.train_val_split(featureSet, labels, val_ratio = .2)

In [ ]:
del train
del featureSet
del labels
gc.collect()

In [ ]:
BATCH_SIZE = 32

In [ ]:
datagen = canaro.generators.imageDataGenerator()
train_generator = datagen.flow(x_train, y_train, batch_size = BATCH_SIZE)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from keras.optimizers import SGD

model = Sequential()
model.add(Input(shape=(80, 80, 1)))

model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2)) 
  
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu')) 
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(256, (3, 3), activation='relu'))  
model.add(Conv2D(256, (3, 3), activation='relu'))  
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu')) 
model.add(Dropout(0.3))
model.add(Dense(len(characters), activation='softmax'))  

optimizer = SGD(learning_rate=0.001, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model.summary()

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler 
callbacks_list = [LearningRateScheduler(canaro.lr_schedule)]

In [ ]:
training = model.fit(
    train_generator, 
    steps_per_epoch = len(x_train) // BATCH_SIZE,
    epochs = 50, 
    validation_data = (x_val, y_val),
    validation_steps = len(y_val) // BATCH_SIZE, 
    callbacks = callbacks_list
)

In [ ]:
characters

# Testing

In [ ]:
test_path = r'/kaggle/input/the-simpsons-characters-dataset/kaggle_simpson_testset/kaggle_simpson_testset/bart_simpson_18.jpg'
img = cv.imread(test_path)

plt.imshow(img)
plt.show()

def prepare(img):
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img = cv.resize(img, (80, 80))
    img = img.astype("float32") / 255.0           
    img = np.reshape(img, (80, 80, 1))            
    img = np.expand_dims(img, axis=0)              
    return img

In [ ]:
predictions = model.predict(prepare(img))
index = np.argmax(predictions)
name = characters[index]

print(f"Predicted character: {name} (index {index})")